# Quantum Testproblem 


Problem from:
https://doi.org/10.1016/j.jcp.2019.04.006

The Problem depends on complex variables that are writen down in a matrix shape. The implementation of the RK-Method is only capable of processing real values. To ensure this we split the values of rho and rhs(rho) in the real and imaginary parts and store theses as a vector. 

We know that the diagonal elemens of rho reamin real. (At least in theory, we should check if this property is destroyed by the commputational inaccuracies) These values should stay in $[0,1]$. It is enougth to ensure $\rho_{n,n} \geq 0 $. 
To implement this into the framework of the used code we generate a vector with the minumum values. We use 

$$ min = \begin{cases}
    0       & \quad \text{if it corresponds to the real value of a diagonal element}\\
    -\infty  & \quad \text{else}
  \end{cases}$$
  
As maximum value we use $+\infty$, which means that there are no constriants

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nodepy import rk
import cvxpy as cp


import numpy.linalg as linalg
import scipy as sci

from numba import jit, float64, stencil

fe =rk.loadRKM('FE').__num__()
rk4 = rk.loadRKM('RK44').__num__()
rk4x2 = rk4*rk4
ssp2 = rk.loadRKM('SSP22').__num__()
ssp3 = rk.loadRKM('SSP33').__num__()
ssp104 = rk.loadRKM('SSP104').__num__()
merson4 = rk.loadRKM('Merson43').__num__()
bs5 = rk.loadRKM('BS5').__num__()

ck5 = rk.loadRKM('CK5').__num__()
dp5 = rk.loadRKM('DP5').__num__()
pd8 = rk.loadRKM('PD8').__num__()

trbdf = rk.loadRKM('TR-BDF2').__num__()
be = rk.loadRKM('BE').__num__()
irk2 = rk.loadRKM('LobattoIIIA2').__num__()



#Extrapolation method
ex2 = rk.extrap(2,'implicit euler').__num__()
ex3 = rk.extrap(3,'implicit euler').__num__()
ex4 = rk.extrap(4,'implicit euler').__num__()
ex5 = rk.extrap(5,'implicit euler').__num__()
ex6 = rk.extrap(6,'implicit euler').__num__()
ex8 = rk.extrap(8,'implicit euler').__num__()

from OrderCondition import *
from RKimple import *
import utils 

In [ ]:
# constants
e0 = 1.60217646e-19
hbar = 1.05457168e-34

# level count
N = 6

# static electric field (V/m)
E = 9e9

# dipole moment
d = 1e-29;

"""
#refernc implementation
# Hamiltonian (diagonal elements)
H = np.zeros([N, N]);
for n in range(1,N): #n = 1:(N - 1)
    H[n,n] = H[n-1, n-1] + (1 - 0.1 * (n - 3)) * 2 * np.pi * 1e13 * hbar;

# Hamiltonian (off-diagonal elements)
for n in range(0,N-1): #n = 1:(N-1)
    H[n, n + 1] = d * E;
    H[n + 1, n] = d * E;
"""

# ωi,i+1 =  ω0[1 − 0.1(i − 3)]

h = np.zeros(N)
for i in range(1,N):
    h[i] = h[i-1] + (1-0.1*(i - 3)) * 2 * np.pi * 1e13 * hbar

H =  np.diag(h) +np.diag(d * E*np.ones(N-1),+1) +np.diag(d * E*np.ones(N-1),-1)



rhs = lambda rho: ( -1j/hbar * (H @ rho - rho @ H))

u0 = np.zeros([N, N]);
u0[0, 0] = 1;
B = u0.reshape(N**2)
u0_vec = np.concatenate((B.real,B.imag))


te = 2.5e-12;
dt = 1e-16;

In [ ]:
#Writing it down in a compatible form 
#we need real values as a vector. For this we reahape the arraz first and then split it into real and imaginary part




def f_quant(t,u):
    N = int(np.sqrt(len(u)/2))
    rho = (u[:N**2]+1j*u[N**2:]).reshape(N,N)
    A = rhs(rho)
    B = A.reshape(N**2)
    return np.concatenate((B.real,B.imag))


def vec2mat(vec):
    N = int(np.sqrt(len(vec)/2))
    return (vec[:N**2]+1j*vec[N**2:]).reshape(N,N)

def mat2vec(mat):
    N = mat.shape[0]
    B = mat.reshape(N**2)
    return np.concatenate((B.real,B.imag))

#A = np.array([[1+1j,0+1j],[0,2+2j]])
#B = A.reshape(4)
#vec = np.concatenate((B.real,B.imag))
#A = (vec[:4]+1j*vec[4:]).reshape(2,2) 
#A
#int(np.sqrt(len(vec)/2))

def extract_populations(u):
    #Function to extract the real values of the diagional elements which represent the populations
    if len(u.shape) == 1: #to make it work with vectors
        u.shape = (u.shape[0],1)
    N = int(np.sqrt(u.shape[0]/2))
    ind = np.arange(N)*(N+1)
    pop = u[ind,:]
    return pop
    
def generate_mainval(N):
    minval = np.repeat(-np.infty,2*N**2)  
    ind = np.arange(N)*(N+1)
    minval[ind] = 0
    return minval

def calculate_ref(u0,dt,t_end):
    N = u0.shape[0]
    rho = u0.copy()
    t_ref = np.arange(0,t_end,dt)
    pop_ref = np.zeros((N,len(t_ref)))
    pop_ref[:, 0] = np.diag(rho).real;
    
    U = sci.linalg.expm(-1j * dt/hbar * H);
    for i in range(1,len(t_ref)):
        rho = U @ rho @ U.conj().transpose();
        pop_ref[:, i] = np.diag(rho).real;
    return t_ref,pop_ref

In [ ]:
t_ref,pop_ref = calculate_ref(u0,4e-16,5.e-13)

## Explicit

In [ ]:
solver = Solver(rkm = dp5,
               dt = 2.e-16,
               t_final = 5.e-13,
               b_fixed=False,
               tol_neg= 1e-8,
               tol_change = 54465283431670.84,
               p = [4,3,2],
               theta = [1,0.5,0.001],
               solver = cp.MOSEK,
               LP_opts = {'reduce':False,'verbose_LP':True})

problem_Q = Problem(f=f_quant,
                 u0=mat2vec(u0),
                 minval=generate_mainval(N),
                 maxval=np.inf)

status,t,u,b,KK = RK_integrate(solver=solver,problem=problem_Q,verbose=False,dumpK=True)


t = np.array(t)
u = np.array(u).T
b = np.array(b).T
utils.show_status(status)

In [ ]:
pop = extract_populations(u)
#plt.plot(t,pop[0,:],label='1')
plt.plot(t,pop[1,:],label='2')
#plt.plot(t,pop[2,:],label='3')
#plt.plot(t,pop[3,:],label='4')
#plt.plot(t,pop[4,:],label='5')
#plt.plot(t,pop[5,:],label='6')

plt.plot(t_ref,pop_ref[1,:],label='2_ref')
plt.grid()
plt.legend()

#plt.ylim([-0.01,0.01])
#plt.xlim([4e-13,5e-13])

In [ ]:
plt.plot(t,np.sum(pop,axis = 0))
#Trace condition violatedwith dt = 4.2e-16,t_final = 5.e-13,

In [ ]:
np.min(pop)

In [ ]:
plt.plot(t[1:],b.T[1:,:]);

## Implcit

In [ ]:
solver = Solver(rkm = ex3,
               dt = 1.0e-16,
               t_final = 1.e-13,
               b_fixed=False,
               tol_neg= 1e-8,
               tol_change = np.inf,
               p = [3,2,1],
               theta = [1],
               solver = cp.ECOS,
               solver_eqs=solver_nonlinear_arg,
               LP_opts = {'reduce':False,'verbose_LP':True})

problem_Q = Problem(f=f_quant,
                 u0=mat2vec(u0),
                 minval=generate_mainval(6),
                 maxval=np.inf)

status,t,u,b = RK_integrate(solver=solver,problem=problem_Q,verbose=False)


t = np.array(t)
u = np.array(u).T
b = np.array(b).T
utils.show_status(status)

In [ ]:
pop = extract_populations(u)
#plt.plot(t,pop[0,:],label='1')
#plt.plot(t,pop[1,:],label='2')
plt.plot(t,pop[2,:],label='3')
#plt.plot(t,pop[3,:],label='4')
#plt.plot(t,pop[4,:],label='5')
#plt.plot(t,pop[5,:],label='6')

plt.plot(t_ref,pop_ref[2,:],label='3_ref')
plt.grid()
plt.legend()

In [ ]:
np.min(pop)

### Additional Code for investigating failed steps

In [ ]:
u0 @ None

In [ ]:
K = KK[-1]

In [ ]:
plt.imshow(K*6e-16)
plt.colorbar()

In [ ]:
np.diag(vec2mat(u[:,-1]+6e-16*K@ssp104.b))

In [ ]:
display(K[0,:])
display(K[7,:])

In [ ]:
(K[0,:]@K[7,:])/(np.linalg.norm(K[0,:])*np.linalg.norm(K[7,:]))

In [ ]:
u[(0,7),-1]

In [ ]:
change = np.array(status['change'])

In [ ]:
np.max(change[change!=None])